# Calculando a pressão de vapor

De acordo com o [artigo do grupo do professo Stephan Grimme](https://pubs.rsc.org/en/content/articlehtml/2022/em/d2em00271j), a pressão de vapor pode ser calculada usando a equação de Ben-Naimm.

\begin{equation} P=\frac{R\times T\times \rho_{liq}}{MW}\times exp\left(\frac{1}{R\times T}\times(\overline{G^*}_{liq}-\overline{G^*}_{vap})\right) \end{equation}
  
| Variável | Descrição | Unidade |
|---|---|---|
| P | pressão de vapor | Pascal (PA) |
| R | constante dos gases | J/(mol*K) |
| T | temperatura | K |
| p_liq | densidade do líquido | kg/m^3 |
| MW | massa molar | kg/mol |
| G_liq | energia livre da fase líquida | J/mol |
| G_vap | energia livre da fase vapor | J/mol |

# Calculando a partir de um csv

O input esperado para o programa é um arquivo no formato CSV com as seguintes colunas.
  
| Nome da coluna | Descrição | Unidade |
|---|---|--|
| Molecula | O nome da molécula | - |
| Comentario | O comentário que você quiser adicionar | - |
| R | constante dos gases | J/(mol*K) |
| T | temperatura | K |
| p_liq | densidade do líquido | kg/m^3 |
| MW | massa molar | kg/mol |
| G_vap | energia livre da fase vapor | Hartree |
| G_liq | energia livre da fase líquida | Hartree |
  
* Os valores nas colunas `G_vap` e `G_liq` estão em Hartree, que é a unidade em que o CENSO retorna esses resultados, ou seja, não precisa converter antes para J/mol, a conversão é feita durante a execução do programa.
* Por padrão, o nome do arquivo que o programa procura é `csv/values.csv`, mas isso você pode mudar abaixo na variável `input_file`.

O programa irá salvar um arquivo denominado `csv/results.csv`, esse arquivo possui as mesmas colunas que o arquivo de entrada, com adição de mais duas colunas.
  
| Nome da coluna | Descrição | Unidade |
|---|---|--|
| PV | Pressão de vapor | Pascal (PA) |
| log_PV | O log na base 10 da pressão de vapor | - |

Abaixo, são calculadas algumas métricas.  
O script pega as moléculas com o mesmo nome na coluna "Moleculas" e calcula a **média** e o **desvio padrão amostral** delas.
* Para a PV (pressão de vapor) são geradas 3 colunas.
> * **count**: número de replicatas.
> * **mean**: média das replicatas.
> * **std**: desvio padrão das replicatas.

* Para o log_PV (logarítmo na base 10 da pressão de vapor) são geradas 2 colunas.
> * **mean**: média das replicatas.
> * **std**: desvio padrão das replicatas.
  
* A tabela gerada é salva como o arquivo `csv/metrics.csv`, e você pode considerar ela como o resultado final.

### Importando as bibliotecas

In [1]:
try:
    import pandas as pd
    import numpy as np
except:
    import os
    os.system('pip install pandas numpy')
    import pandas as pd
    import numpy as np

### Definindo as funções

In [2]:
def hartree_to_j_mol(x):
    """
    Convert Hartree to J/mol.
    Args:
        x: value in Hartree
    Returns:
        value in J/mol
    """
    return x * 2625.5 * 1000


def vapor_pressure(row):
    """
    Calculate vapor pressure using the Ben-Naim equation of state.
    Args:
        R: gas constant (in J/(mol*K))
        T: temperature (in K)
        p_liq: liquid density (in kg/m^3)
        MW: molecular weight (in kg/mol)
        G_liq: free energy of liquid (in Hartree)
        G_vap: free energy of vapor (in Hartree)
    Returns:
        vapor pressure (in Pa)
    """

    R = float(row["R"])
    T = float(row["T"])
    p_liq = float(row["p_liq"])
    MW = float(row["MW"])
    G_liq = float(hartree_to_j_mol(row["G_liq"]))
    G_vap = float(hartree_to_j_mol(row["G_vap"]))

    a = (R * T * p_liq) / (MW)
    b = np.exp((G_liq - G_vap) / (R * T))
    return a * b


def calculate_vapor_pressure(input_file, output_file):
    """
    Run the calculation of vapor pressure.

    Args:
        input_file: path to the input file
        output_file: path to the output file
    Returns:
        df: dataframe with the calculated vapor pressure
    """

    # Open the file
    df = pd.read_csv(input_file)
    print(f"* Arquivo de entrada {input_file} aberto com sucesso!")

    # Calculate vapor pressure
    df["PV"] = df.apply(vapor_pressure, axis=1)

    # Calculate log of vapor pressure
    df["log_PV"] = np.log10(df["PV"])
    
    print("* Pressão de vapor calculada com sucesso!")

    # Save the file
    df.to_csv(output_file, index=False)
    print(f"* Arquivo de saída {output_file} que contém os resultados salvo com sucesso!")

    print("* Imprimindo as pressões de vapor calculadas!")
    display(df[["Molecula", "Comentario", "PV", "log_PV"]].dropna(how="all"))

    print(f"* obs: no arquivo {output_file} os valores estão salvos com todas as casas decimais.")

    return df.dropna(how="all")


def generate_metrics(df, output_file="metrics.csv"):
    """
    Generate metrics for the calculated vapor pressure.
    Args:
        df: dataframe with the calculated vapor pressure
    Returns:
        metrics: dataframe with the calculated metrics
    """
    # Calculate metrics
    print(f"* Calculando as métricas!")
    metrics = df.groupby("Molecula").agg({"PV": {"mean", "std", "count"}, "log_PV": {"mean", "std"}})
    print(f"* Métricas calculadas com sucesso!")

    # Save metrics
    metrics.to_csv(output_file)
    print(f"* Arquivo {output_file} contendo as métricas salvo com sucesso!")

    # Print metrics
    print("* Imprimindo as métricas!")
    display(metrics)

### Calculando a **Pressão de Vapor**

In [3]:
# Define the input and output filename
input_file = "values.csv"
output_file = "results.csv"

if __name__ == "__main__":
    df = calculate_vapor_pressure(input_file=input_file, output_file=output_file)

* Arquivo de entrada values.csv aberto com sucesso!
* Pressão de vapor calculada com sucesso!
* Arquivo de saída results.csv que contém os resultados salvo com sucesso!
* Imprimindo as pressões de vapor calculadas!


/home/lucas/.local/share/virtualenvs/CRENSO-CyzZ7mNt/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Molecula,Comentario,PV,log_PV
0,GLY,NaN,1.642823e-02,-1.784409
1,GLY_corr,NaN,1.903383e-02,-1.720474
2,c10,NaN,2.997594e+02,2.476773
3,c10_corr,NaN,2.931660e+02,2.467114
4,nap,NaN,2.072197e+01,1.316431
5,ABA,replicata 1,9.348678e-06,-5.029250
6,ABA_corr,replicata 1,1.180166e-05,-4.928057
7,ABA,replicata 2,5.928763e-06,-5.227036
8,ABA_corr,replicata 2,6.398553e-06,-5.193918
9,ABA,replicata 3,8.026290e-06,-5.095485


* obs: no arquivo results.csv os valores estão salvos com todas as casas decimais.


### Gerando métricas
* O script abaixo pega as moléculas com o mesmo nome na coluna "Moleculas" e calcula a **média** e o **desvio padrão amostral** delas.
* Para a PV (pressão de vapor) são geradas 3 colunas.
> * **count**: número de replicatas.
> * **mean**: média das replicatas.
> * **std**: desvio padrão das replicatas.

* Para o log_PV (logarítmo na base 10 da pressão de vapor) são geradas 2 colunas.
> * **mean**: média das replicatas.
> * **std**: desvio padrão das replicatas.

In [4]:
# Define the output filename
output_file = "metrics.csv"

if __name__ == "__main__":
    generate_metrics(df=df, output_file=output_file)

* Calculando as métricas!
* Métricas calculadas com sucesso!
* Arquivo metrics.csv contendo as métricas salvo com sucesso!
* Imprimindo as métricas!


PV                            log_PV          
         count       std          mean       std      mean
Molecula                                                  
ABA          3  0.000002  7.767911e-06  0.100674 -5.117257
ABA_corr     3  0.000003  8.881697e-06  0.133124 -5.065127
DET          1       NaN  5.863751e-02       NaN -1.231825
DET_corr     1       NaN  5.109515e-02       NaN -1.291620
DIN          1       NaN  8.816532e-02       NaN -1.054702
DIN_corr     1       NaN  3.115869e-01       NaN -0.506421
GLY          1       NaN  1.642823e-02       NaN -1.784409
GLY_corr     1       NaN  1.903383e-02       NaN -1.720474
MEF          1       NaN  1.537168e-07       NaN -6.813279
MEF_corr     1       NaN  9.255436e-08       NaN -7.033603
PAV          1       NaN  0.000000e+00       NaN      -inf
PAV_corr     1       NaN  0.000000e+00       NaN      -inf
TKT          1       NaN  0.000000e+00       NaN      -inf
TKT_corr     1       NaN  0.000000e+00       NaN      -inf
c10          1       NaN  2.997594e+02       NaN  2.476773
c10_corr     1       NaN  2.931660e+02       NaN  2.467114
nap          1       NaN  2.072197e+01       NaN  1.316431